In [1]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv
import trimesh
from copy import deepcopy
%matplotlib widget

### Create a set of equidistant points for a sphere

In [2]:
# Create a range of equidistant points on a sphere using Deserno 2004,
# but with correction that points are first determined on unit sphere, 
# then expanded by conversion to cartesian coordinates

r = 1 #radius in mm
N=5000 #goal of number points
sphere_points=[]
n_count = 0 # actual number of points


#a = 4*np.pi*r**2./ N
a = 4*np.pi / N
d = np.sqrt(a)
M_theta = np.round(np.pi/d)
d_theta = np.pi/M_theta
d_phi = a/d_theta
for m in np.arange(0,M_theta):
    theta = np.pi*(m+0.5) / M_theta
    M_phi = np.round(2*np.pi*np.sin(theta)/d_phi)
    for n in np.arange(0,M_phi):
        phi = 2*np.pi*n/M_phi
        x = r * np.sin(theta)*np.cos(phi)
        y = r * np.sin(theta)*np.sin(phi)
        z = r * np.cos(theta)
        sphere_points.append((x,y,z))
        n_count += 1

print (n_count)
sphere_points = np.array(sphere_points)

4999


### Add equally spaced lobes to the sphere surface

In [3]:
n_lobes = 50 #goal number of lobes
lobe_centers = [] #just the center points
lobes = [] #successive sets of points, including circles around center points, one for each lobe
lobe_count = 0 #actual number of lobes
lobe_thetas = []
lobe_phis = []

#get the spherical coordinates (on a unit sphere) of all lobe centers
#basically, repeat the stuff in previous cell but for fewer points
a = 4*np.pi / n_lobes
d = np.sqrt(a)
M_theta = np.round(np.pi/d)
#d_theta = np.pi/M_theta
d_theta = 0.95*np.pi/M_theta #seemed to crowd at the poles, so this pulls it away a bit more
d_phi = a/d_theta
for m in np.arange(0,M_theta):
    #theta = np.pi*(m+0.5) / M_theta
    theta = 0.95*np.pi*(m+0.55) / M_theta
    #theta = np.pi*(m+0.55) / M_theta
    M_phi = np.round(2*np.pi*np.sin(theta)/d_phi)
    for n in np.arange(0,M_phi):
        phi = 2*np.pi*n/M_phi
        lobe_thetas.append(theta)
        lobe_phis.append(phi)
        x = r * np.sin(theta)*np.cos(phi)
        y = r * np.sin(theta)*np.sin(phi)
        z = r * np.cos(theta)
        lobe_centers.append((x,y,z))
        lobe_count += 1

#now calculate circles around each lobe point, at a central angle of alpha
# following https://math.stackexchange.com/questions/643130/circle-on-sphere
# Ah ha! But note their phi and theta angles are switched from the Deserno 2004 paper
alpha_start = 0.45*np.minimum(d_theta, d_phi)
radius = 1. #set to sphere radius to start
num_t_at_start = 200
circ_at_start = 2*np.pi*alpha_start
dt = circ_at_start / num_t_at_start

for phi, theta, lobe_center in zip(lobe_phis, lobe_thetas, lobe_centers):
    beta = theta
    gamma = np.pi - phi
    lobe_points = []
    
    #cycle through successively smaller alphas, setting the radius/height increasingly higher
    # each time
    alphas = np.linspace(alpha_start, 0, 200)
    #lobe_heights = radius + np.sin(np.pi/2 * np.linspace(0,0.5,10)) #equation too sharp
    lobe_heights = radius + -(np.cos(np.pi*np.linspace(0,1.0,200))-1)/4  #followed https://easings.net/#easeInOutSine
    first_time_through = True
    for alpha, lobe_height in zip(alphas[:-1], lobe_heights[:-1]):
    
        #calculate number of t needed for this alpha
        circ_here = 2*np.pi*alpha
        num_t = int(np.round(circ_here/dt))
        t = np.linspace(0,2*np.pi,num_t)
        #print (alpha, circ_here, num_t)

        #cartesian coordinates for each point in the circle 
        xs = lobe_height*(np.sin(alpha)*np.cos(beta)*np.cos(gamma)*np.cos(t) + 
                            np.sin(alpha)*np.sin(gamma)*np.sin(t)-
                            np.cos(alpha)*np.sin(beta)*np.cos(gamma))
        ys = lobe_height*(-np.sin(alpha)*np.cos(beta)*np.sin(gamma)*np.cos(t) + 
                            np.sin(alpha)*np.cos(gamma)*np.sin(t)+
                            np.cos(alpha)*np.sin(beta)*np.sin(gamma))
        zs = lobe_height*(np.sin(alpha)*np.sin(beta)*np.cos(t)+np.cos(alpha)*np.cos(beta))
        
        # if first time through, calculate great circle distance from center point to all points in circle
        # (should all be the same)
        if first_time_through:
            #gcd between circle and center point on unit sphere is just the angle between them - alpha!
            #cartesian coordinates calculation if you want to prove it, below
            # gcd = []
            # for thisx,thisy,thisz in zip(xs,ys,zs):
            #     a = np.array((thisx,thisy,thisz))
            #     b = np.array(lobe_centers[0])
            #     a_dot_b = np.dot(a,b)
            #     angle = np.arccos(a_dot_b / ( (a[0]**2+a[1]**2+a[2]**2)**0.5 * (b[0]**2+b[1]**2+b[2]**2)**0.5 ) )
            #     gcd.append(r*angle)
            gcd = alpha
            
            #now, determine which points in sphere points array are closer to center point than that.
            #remove them, as they will become "inside" points underneath the lobe once surfaces are merged.
            a = sphere_points
            b = np.array(lobe_center)
            a_dot_b = np.dot(a,b)
            angle = np.arccos(a_dot_b / ( (a[:,0]**2+a[:,1]**2+a[:,2]**2)**0.5 * (b[0]**2+b[1]**2+b[2]**2)**0.5 ) )
            points_gcds = r*angle
            inside_circle = points_gcds < gcd
            sphere_points = np.delete(sphere_points,inside_circle,axis=0)

        #last point is right at alpha = 0
        alpha = alphas[-1]
        lobe_height = lobe_heights[-1]
        xs = np.append(xs, lobe_height*(np.sin(theta)*np.cos(phi))) #use the x,y,z location of lobe center but add lobe_height
        ys = np.append(ys, lobe_height*(np.sin(theta)*np.sin(phi)))
        zs = np.append(zs, lobe_height*(np.cos(theta)))
        
        dum = [lobe_points.append((i,j,k)) for i,j,k in zip(xs,ys,zs)]
        first_time_through = False
    
    lobes.append(lobe_points)

print (len(lobes))
print (len(lobe_centers))
print (sphere_points.shape)



51
51
(1785, 3)


### Reconstruct a surface from points in the point cloud

In [4]:
lobe_centers = np.array(lobe_centers)
lobe_cloud = pv.PolyData(lobe_centers)
sphere_cloud = pv.PolyData(sphere_points)


#calculate a separate surface for each lobe
lobe_surfs = []
#all_lobe_points = []
for lobe in lobes:
    #all_lobe_points = all_lobe_points+lobe
    lobe_points = np.array(lobe)
    lobe_point_cloud = pv.PolyData(lobe_points)
    lobe_surfs.append(lobe_point_cloud.reconstruct_surface(sample_spacing=0.05))

#make a point cloub of all the lobe points
#all_lobe_points = np.array(all_lobe_points)
#lobe_point_cloud = pv.PolyData(all_lobe_points)

#add our surfaces together
sphere_surf = sphere_cloud.reconstruct_surface()
total_surf = deepcopy(sphere_surf)
for lobe_surf in lobe_surfs:
    total_surf = total_surf + lobe_surf 

#and plot, by individual lobe and then combined surface    
pv.global_theme.color_cycler = 'default'
pl = pv.Plotter(shape=(1, 2))
pl.subplot(0, 0)
_ = pl.add_mesh(sphere_surf)
#_ = pl.add_mesh(lobe_point_cloud,color='k')
for lobe_surf in lobe_surfs:
    _ = pl.add_mesh(lobe_surf)
    
pl.subplot(0, 1)
_ = pl.add_mesh(total_surf,color='blue')
pl.show()



Widget(value='<iframe src="http://localhost:57727/index.html?ui=P_0x17ab90710_0&reconnect=auto" class="pyvista…

### Write mesh out to STL file

In [7]:
total_surf.save('equimesh_50lobe.stl')


#now plot again
mesh = trimesh.load_mesh("equimesh_50lobe.stl")
# Show the mesh (opens in a window)
mesh.show()


In [8]:
#mesh quality
print(f"- Is watertight: {mesh.is_watertight}")
print(f"- Volume: {mesh.volume:.4f}")
print(f"- Surface area: {mesh.area:.4f}")




- Is watertight: False
- Volume: -6.3381
- Surface area: 45.1668
